# Libraries

In [ ]:
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Flatten, Input
from tensorflow.keras.utils import to_categorical
#from keras.utils.np_utils import to_categorical
from keras.layers import Conv2D, Dropout, MaxPooling2D
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator, img_to_array, array_to_img
from keras.utils import to_categorical
from tensorflow.keras import layers

import zipfile
import subprocess
import pandas as pd
import numpy as np
import cv2
import os

# Load the Data

In [ ]:
!pip install -U -q kaggle
!mkdir -p ~/.kaggle
from google.colab import files
files.upload()

In [ ]:
!cp kaggle.json ~/.kaggle/
!kaggle datasets download -d gpiosenka/yikes-spiders-15-species

yikes-spiders-15-species.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
zip_file = "/content/yikes-spiders-15-species.zip"
destination_dir = "/content/archive"

unzip_process = subprocess.Popen(["unzip", zip_file, "-d", destination_dir])

unzip_process.wait()

1

# Data Preprocessing

In [ ]:
folders = sorted(os.listdir("/content/archive/train/"))
folders

['Black Widow',
 'Blue Tarantula',
 'Bold Jumper',
 'Brown Grass Spider',
 'Brown Recluse Spider',
 'Deinopis Spider',
 'Golden Orb Weaver',
 'Hobo Spider',
 'Huntsman Spider',
 'Ladybird Mimic Spider',
 'Peacock Spider',
 'Red Knee Tarantula',
 'Spiny-backed Orb-weaver',
 'White Kneed Tarantula',
 'Yellow Garden Spider']

In [ ]:
# Classes
c_spider = {
    'Black Widow':0,
    'Blue Tarantula':1,
    'Bold Jumper':2,
    'Brown Grass Spider':3,
    'Brown Recluse Spider':4,
    'Deinopis Spider':5,
    'Golden Orb Weaver':6,
    'Hobo Spider':7,
    'Huntsman Spider':8,
    'Ladybird Mimic Spider':9,
    'Peacock Spider':10,
    'Red Knee Tarantula':11,
    'Spiny-backed Orb-weaver':12,
    'White Kneed Tarantula':13,
    'Yellow Garden Spider':14
}

#Hyperclasses     im=1, nim=0
m_importance = {
    'Black Widow':1,
    'Blue Tarantula':0,
    'Bold Jumper':0,
    'Brown Grass Spider':0,
    'Brown Recluse Spider':1,
    'Deinopis Spider':0,
    'Golden Orb Weaver':0,
    'Hobo Spider':0,
    'Huntsman Spider':0,
    'Ladybird Mimic Spider':0,
    'Peacock Spider':0,
    'Red Knee Tarantula':0,
    'Spiny-backed Orb-weaver':0,
    'White Kneed Tarantula':0,
    'Yellow Garden Spider':0
}

In [ ]:
ms = {
    'Black Widow' : 0,
    'Brown Recluse Spider' : 1
}

In [ ]:
keys_1 = list(m_importance)

keys_2 = list(c_spider)

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=60,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [ ]:
data_xtrain = []
data_ytrain = []
i=0

for folder in folders:
  if folder == 'Black Widow' or folder == 'Brown Recluse Spider':
    for filename in os.listdir(f"/content/archive/train/{folder}"):
        if filename.endswith("jpg"):
            pic = cv2.imread(f"/content/archive/train/{folder}/{filename}")
            pic = cv2.resize(pic, (32,32))

            data_ytrain.append(1)
            data_xtrain.append(pic)

            x = img_to_array(pic)
            x = x.reshape((1,) + x.shape)
            for batch in datagen.flow(x, batch_size=1):
                augmented_image = array_to_img(batch[0])
                data_xtrain.append(img_to_array(augmented_image))
                data_ytrain.append(1)
                i += 1
                if i >= 3:
                    break

  else:
    for filename in os.listdir(f"/content/archive/train/{folder}"):
        if filename.endswith("jpg"):
            pic = cv2.imread(f"/content/archive/train/{folder}/{filename}")
            pic = cv2.resize(pic, (32,32))
            data_ytrain.append(0)
            data_xtrain.append(pic)

            x = img_to_array(pic)
            x = x.reshape((1,) + x.shape)
            for batch in datagen.flow(x, batch_size=1):
                augmented_image = array_to_img(batch[0])
                data_xtrain.append(img_to_array(augmented_image))
                data_ytrain.append(0)
                i += 1
                if i >= 3:
                    break


X_train = np.array(data_xtrain)

y_train = np.array(data_ytrain)

y_train = to_categorical(y_train)

print(X_train.shape)
print(y_train.shape)

(4372, 32, 32, 3)
(4372, 2)


In [ ]:
X_train = X_train / 255

In [ ]:
data_xtest = []
data_ytest = []

for folder in folders:
  if folder == 'Black Widow' or folder == 'Brown Recluse Spider':
    for filename in os.listdir(f"/content/archive/test/{folder}"):
        if filename.endswith("jpg"):
            pic = cv2.imread(f"/content/archive/test/{folder}/{filename}")
            pic = cv2.resize(pic, (32,32))
            data_ytest.append(1)
            data_xtest.append(pic)

            x = img_to_array(pic)
            x = x.reshape((1,) + x.shape)
            for batch in datagen.flow(x, batch_size=1):
                augmented_image = array_to_img(batch[0])
                data_xtest.append(img_to_array(augmented_image))
                data_ytest.append(1)
                i += 1
                if i >= 3:
                    break
  else:
    for filename in os.listdir(f"/content/archive/test/{folder}"):
        if filename.endswith("jpg"):
            pic = cv2.imread(f"/content/archive/test/{folder}/{filename}")
            pic = cv2.resize(pic, (32,32))
            data_ytest.append(0)
            data_xtest.append(pic)

            x = img_to_array(pic)
            x = x.reshape((1,) + x.shape)
            for batch in datagen.flow(x, batch_size=1):
                augmented_image = array_to_img(batch[0])
                data_xtest.append(img_to_array(augmented_image))
                data_ytest.append(0)
                i += 1
                if i >= 3:
                    break

X_test = np.array(data_xtest)

y_test = np.array(data_ytest)

y_test = to_categorical(y_test)

print(X_test.shape)
print(y_test.shape)

(150, 32, 32, 3)
(150, 2)


In [ ]:
X_test = X_test / 255

In [ ]:
data_xvalid = []
data_yvalid = []

for folder in folders:
  if folder == 'Black Widow' or folder == 'Brown Recluse Spider':
    for filename in os.listdir(f"/content/archive/valid/{folder}"):
        if filename.endswith("jpg"):
            pic = cv2.imread(f"/content/archive/valid/{folder}/{filename}")
            pic = cv2.resize(pic, (32,32))
            data_yvalid.append(1)
            data_xvalid.append(pic)
  else:
    for filename in os.listdir(f"/content/archive/valid/{folder}"):
        if filename.endswith("jpg"):
            pic = cv2.imread(f"/content/archive/valid/{folder}/{filename}")
            pic = cv2.resize(pic, (32,32))
            data_yvalid.append(0)
            data_xvalid.append(pic)

X_valid = np.array(data_xvalid)

y_valid = np.array(data_yvalid)

y_valid = to_categorical(y_valid)

print(X_valid.shape)
print(y_valid.shape)

(75, 32, 32, 3)
(75, 2)


# Models

In [ ]:
!pip install keras

In [ ]:
img_height = 32
img_width = 32
data_augmentation = keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal",
                                                 input_shape=(img_height,img_width,3)),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1),
])

In [ ]:
model_00 = Sequential()
model_00.add(data_augmentation)
model_00.add(Conv2D(32, (3,3), activation="relu", padding="same", input_shape=(32,32,3)))
model_00.add(Conv2D(32, (3,3), activation="relu", padding="same"))
model_00.add(MaxPooling2D((2,2)))
model_00.add(Dropout(0.25))

model_00.add(Conv2D(64, (3,3), activation="relu", padding="same"))
model_00.add(Conv2D(64, (3,3), activation="relu", padding="same"))
model_00.add(MaxPooling2D((2,2)))
model_00.add(Dropout(0.50))

model_00.add(Flatten())
model_00.add(Dense(128, activation='relu'))
model_00.add(Dense(2, activation='softmax'))

model_00.compile(optimizer='Adamax', loss='categorical_crossentropy', metrics=['accuracy'])
historial = model_00.fit(X_train, y_train, epochs=55, batch_size=50, validation_data=(X_test,y_test))

Epoch 1/55
88/88 [==============================] - 3s 12ms/step - loss: 0.3768 - accuracy: 0.8769 - val_loss: 0.3917 - val_accuracy: 0.8667
Epoch 2/55
88/88 [==============================] - 1s 8ms/step - loss: 0.3240 - accuracy: 0.8797 - val_loss: 0.3564 - val_accuracy: 0.8667
Epoch 3/55
88/88 [==============================] - 1s 14ms/step - loss: 0.3159 - accuracy: 0.8797 - val_loss: 0.3709 - val_accuracy: 0.8667
Epoch 4/55
88/88 [==============================] - 1s 15ms/step - loss: 0.3138 - accuracy: 0.8797 - val_loss: 0.3605 - val_accuracy: 0.8667
Epoch 5/55
88/88 [==============================] - 1s 13ms/step - loss: 0.3144 - accuracy: 0.8797 - val_loss: 0.3680 - val_accuracy: 0.8667
Epoch 6/55
88/88 [==============================] - 1s 9ms/step - loss: 0.3121 - accuracy: 0.8797 - val_loss: 0.3740 - val_accuracy: 0.8667
Epoch 7/55
88/88 [==============================] - 1s 8ms/step - loss: 0.3144 - accuracy: 0.8797 - val_loss: 0.3612 - val_accuracy: 0.8667
Epoch 8/55
88/88

In [ ]:
model_00.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 32, 32, 3)         0         
                                                                 
 conv2d_10 (Conv2D)          (None, 32, 32, 32)        896       
                                                                 
 conv2d_11 (Conv2D)          (None, 32, 32, 32)        9248      
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 16, 16, 32)        0         
 g2D)                                                            
                                                                 
 dropout_8 (Dropout)         (None, 16, 16, 32)        0         
                                                                 
 conv2d_12 (Conv2D)          (None, 16, 16, 64)        18496     
                                                      

In [ ]:
model_00.save('web_9090.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
web = keras.models.load_model("web_9090.h5")

In [ ]:
dickeys = ['non-medically significant','medically significant']

In [ ]:
def PredIm(image_name):
  image = cv2.imread(image_name)
  image = cv2.resize(image, (32,32))
  image = np.expand_dims(image,0)
  return print("Bites from this spider species are considered", dickeys[int(np.argmax(web(image),1))])

In [ ]:
PredIm('/content/archive/valid/Black Widow/1.jpg')

Bites from this spider species are considered non-medically significant


# Alternative Aproaches

In [ ]:
model_00 = Sequential()
model_00.add(data_augmentation)
model_00.add(Conv2D(32, (3,3), activation="relu", padding="same", input_shape=(32,32,3)))
model_00.add(Conv2D(32, (3,3), activation="relu", padding="same"))
model_00.add(MaxPooling2D((2,2)))
model_00.add(Dropout(0.25))

model_00.add(Conv2D(64, (3,3), activation="relu", padding="same"))
model_00.add(Dropout(0.25))

model_00.add(Conv2D(128, (3,3), activation="relu", padding="same"))
model_00.add(Conv2D(256, (3,3), activation="relu", padding="same"))
model_00.add(MaxPooling2D((2,2)))
model_00.add(Dropout(0.25))

model_00.add(Flatten())
model_00.add(Dense(512, activation='relu'))
model_00.add(Dropout(0.25))
model_00.add(Dense(2, activation='softmax'))

model_00.compile(optimizer='Adamax', loss='categorical_crossentropy', metrics=['accuracy'])
historial = model_00.fit(X_train, y_train, epochs=55, batch_size=50, validation_data=(X_test,y_test))

Epoch 1/55
88/88 [==============================] - 4s 18ms/step - loss: 0.3962 - accuracy: 0.8733 - val_loss: 0.4479 - val_accuracy: 0.8667
Epoch 2/55
88/88 [==============================] - 1s 15ms/step - loss: 0.3203 - accuracy: 0.8797 - val_loss: 0.4047 - val_accuracy: 0.8667
Epoch 3/55
88/88 [==============================] - 1s 15ms/step - loss: 0.3131 - accuracy: 0.8797 - val_loss: 0.3949 - val_accuracy: 0.8667
Epoch 4/55
88/88 [==============================] - 2s 17ms/step - loss: 0.3161 - accuracy: 0.8797 - val_loss: 0.3924 - val_accuracy: 0.8667
Epoch 5/55
88/88 [==============================] - 2s 21ms/step - loss: 0.3111 - accuracy: 0.8797 - val_loss: 0.3944 - val_accuracy: 0.8667
Epoch 6/55
88/88 [==============================] - 3s 30ms/step - loss: 0.3099 - accuracy: 0.8797 - val_loss: 0.3966 - val_accuracy: 0.8667
Epoch 7/55
88/88 [==============================] - 2s 23ms/step - loss: 0.3077 - accuracy: 0.8797 - val_loss: 0.3855 - val_accuracy: 0.8667
Epoch 8/55
88

In [ ]:
model_00.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 32, 32, 3)         0         
                                                                 
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2  (None, 16, 16, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 16, 16, 32)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 64)        18496     
                                                      

In [ ]:
data_xtrain = []
data_ytrain = []

for folder in folders:
  for filename in os.listdir(f"/content/archive/train/{folder}"):
      if filename.endswith("jpg"):
          pic = cv2.imread(f"/content/archive/train/{folder}/{filename}")
          pic = cv2.resize(pic, (32,32))
          data_ytrain.append([dic[folder]])
          data_xtrain.append(pic)

In [ ]:
X_train = np.array(data_xtrain)
print(X_train.shape)
y_train = np.array(data_ytrain)
print(y_train.shape)

(2185, 32, 32, 3)
(2185, 1)


In [ ]:
y_train = to_categorical(y_train)
print(y_train.shape)

(2185, 15)


In [ ]:
data_xtest = []
data_ytest = []

for folder in folders:
  for filename in os.listdir(f"/content/archive/test/{folder}"):
      if filename.endswith("jpg"):
          pic = cv2.imread(f"/content/archive/test/{folder}/{filename}")
          pic = cv2.resize(pic, (32,32))
          data_ytest.append([dic[folder]])
          data_xtest.append(pic)

In [ ]:
X_test = np.array(data_xtest)
print(X_test.shape)
y_test = np.array(data_ytest)
print(y_test.shape)

(75, 32, 32, 3)
(75, 1)


In [ ]:
y_test = to_categorical(y_test)
print(y_test.shape)

(75, 15)


In [ ]:
data_xvalid = []
data_yvalid = []

for folder in folders:
  for filename in os.listdir(f"/content/archive/valid/{folder}"):
      if filename.endswith("jpg"):
          pic = cv2.imread(f"/content/archive/valid/{folder}/{filename}")
          pic = cv2.resize(pic, (32,32))
          data_yvalid.append([dic[folder]])
          data_xvalid.append(pic)

In [ ]:
X_valid = np.array(data_xvalid)
print(X_valid.shape)
y_valid = np.array(data_yvalid)
print(y_valid.shape)

(75, 32, 32, 3)
(75, 1)


In [ ]:
y_valid = to_categorical(y_valid)
print(y_valid.shape)

(75, 15)


In [ ]:
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Flatten, Input
from keras.utils.np_utils import to_categorical
from keras.layers import Conv2D, Dropout, MaxPooling2D

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3,3), activation="relu", padding="same", input_shape=(32,32,3)))
model.add(Conv2D(32, (3,3), activation="relu", padding="same"))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3,3), activation="relu", padding="same"))
model.add(Conv2D(64, (3,3), activation="relu", padding="same"))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3,3), activation="relu", padding="same"))
model.add(Conv2D(128, (3,3), activation="relu", padding="same"))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(15, activation='softmax'))

#model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
#historial = model.fit(X_train, y_train, epochs=60, batch_size=128, validation_data=(X_test,y_test))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 16, 16, 32)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 64)        18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 16, 16, 64)        36928     
                                                        

In [ ]:
#model.save("SpiderClassifier.h5")

In [ ]:
#RedQuePredice=keras.models.load_model("SpiderClassifier.h5")

In [ ]:
#print("The image is most likely to be a: ", dickeys[int(np.argmax(RedQuePredice.predict(viuda),1))])

In [ ]:
def PredIm(image_name):
  image = cv2.imread(image_name)
  image = cv2.resize(image, (32,32))
  image = np.expand_dims(image,0)
  return print("It is most likely to be about a:", dickeys[int(np.argmax(RedQuePredice(image),1))])

In [ ]:
#PredIm("reclusa.jpg")

In [ ]:
model02 = Sequential()
model02.add(Conv2D(32, (3,3), activation="relu", padding="same", input_shape=(32,32,3)))
model02.add(Conv2D(32, (3,3), activation="relu", padding="same"))
model02.add(MaxPooling2D((2,2)))
model02.add(Dropout(0.25))

model02.add(Conv2D(64, (3,3), activation="relu", padding="same"))
model02.add(Dropout(0.25))

model02.add(Conv2D(128, (3,3), activation="relu", padding="same"))
model02.add(Conv2D(256, (3,3), activation="relu", padding="same"))
model02.add(Dropout(0.25))

model02.add(Flatten())
model02.add(Dense(512, activation='relu'))
model02.add(Dropout(0.25))
model02.add(Dense(15, activation='softmax'))

model02.compile(optimizer='Adamax', loss='categorical_crossentropy', metrics=['accuracy'])
historial = model02.fit(X_train, y_train, epochs=55, batch_size=50, validation_data=(X_test,y_test))

Epoch 1/55
44/44 [==============================] - 4s 32ms/step - loss: 36.8130 - accuracy: 0.0682 - val_loss: 2.7036 - val_accuracy: 0.0800
Epoch 2/55
44/44 [==============================] - 1s 19ms/step - loss: 2.6976 - accuracy: 0.0815 - val_loss: 2.6718 - val_accuracy: 0.0667
Epoch 3/55
44/44 [==============================] - 1s 20ms/step - loss: 2.6620 - accuracy: 0.0906 - val_loss: 2.5015 - val_accuracy: 0.1467
Epoch 4/55
44/44 [==============================] - 1s 20ms/step - loss: 2.5352 - accuracy: 0.1483 - val_loss: 2.2181 - val_accuracy: 0.2667
Epoch 5/55
44/44 [==============================] - 1s 19ms/step - loss: 2.3781 - accuracy: 0.2082 - val_loss: 1.9234 - val_accuracy: 0.3867
Epoch 6/55
44/44 [==============================] - 1s 19ms/step - loss: 2.2664 - accuracy: 0.2540 - val_loss: 1.8003 - val_accuracy: 0.4133
Epoch 7/55
44/44 [==============================] - 1s 19ms/step - loss: 2.1362 - accuracy: 0.2938 - val_loss: 1.8078 - val_accuracy: 0.4133
Epoch 8/55
4

In [ ]:
model02.save("SpiderClassifier96-73.h5")

In [ ]:
model03 = Sequential()
model03.add(Conv2D(32, (3,3), activation="relu", padding="same", input_shape=(32,32,3)))
model03.add(Conv2D(32, (3,3), activation="relu", padding="same"))
model03.add(MaxPooling2D((2,2)))
model03.add(Dropout(0.25))

model03.add(Conv2D(64, (3,3), activation="relu", padding="same"))
model03.add(Conv2D(64, (3,3), activation="relu", padding="same"))
model03.add(Dropout(0.25))

model03.add(Conv2D(128, (3,3), activation="relu", padding="same"))
model03.add(Conv2D(256, (3,3), activation="relu", padding="same"))
model03.add(Dropout(0.25))

model03.add(Flatten())
model03.add(Dense(512, activation='relu'))
model03.add(Dropout(0.25))
model03.add(Dense(15, activation='softmax'))

model03.compile(optimizer='Adamax', loss='categorical_crossentropy', metrics=['accuracy'])
historial = model03.fit(X_train, y_train, epochs=100, batch_size=128, validation_data=(X_test,y_test))

Epoch 1/100
18/18 [==============================] - 3s 51ms/step - loss: 25.2820 - accuracy: 0.0728 - val_loss: 2.7082 - val_accuracy: 0.0667
Epoch 2/100
18/18 [==============================] - 1s 35ms/step - loss: 2.7060 - accuracy: 0.0732 - val_loss: 2.6940 - val_accuracy: 0.0667
Epoch 3/100
18/18 [==============================] - 1s 35ms/step - loss: 2.6849 - accuracy: 0.1066 - val_loss: 2.6297 - val_accuracy: 0.1733
Epoch 4/100
18/18 [==============================] - 1s 35ms/step - loss: 2.6346 - accuracy: 0.1455 - val_loss: 2.5339 - val_accuracy: 0.1733
Epoch 5/100
18/18 [==============================] - 1s 35ms/step - loss: 2.5835 - accuracy: 0.1446 - val_loss: 2.4146 - val_accuracy: 0.2267
Epoch 6/100
18/18 [==============================] - 1s 35ms/step - loss: 2.5298 - accuracy: 0.1638 - val_loss: 2.3465 - val_accuracy: 0.2667
Epoch 7/100
18/18 [==============================] - 1s 35ms/step - loss: 2.4527 - accuracy: 0.1899 - val_loss: 2.2206 - val_accuracy: 0.2800
Epoch

In [ ]:
from keras.saving.saving_lib import load_model
red = keras.models.load_model("SpiderClassifier96-73.h5")

In [ ]:
def PredIm(image_name):
  image = cv2.imread(image_name)
  image = cv2.resize(image, (32,32))
  image = np.expand_dims(image,0)
  return print("It is most likely to be about a:", dickeys[int(np.argmax(red(image),1))])

In [ ]:
PredIm("viuda10.jpg")

It is most likely to be about a: Golden Orb Weaver
